In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# Initialize Spark session
spark = SparkSession.builder.appName("CRUD Operations with TempView").getOrCreate()

# Manually creating a DataFrame using Row
data = [
    Row(ID=1, Name="Alice", Age=25),
    Row(ID=2, Name="Bob", Age=30),
    Row(ID=3, Name="Charlie", Age=35)
]

# Create DataFrame
df_manual = spark.createDataFrame(data)

# Create a temporary view for SQL queries
df_manual.createOrReplaceTempView("people")

# Select records where Age is greater than 25
filtered_records = spark.sql("SELECT * FROM people WHERE Age > 25")
filtered_records.show()

# Insert new records
new_data = [
    Row(ID=4, Name="David", Age=40),
    Row(ID=5, Name="Eva", Age=28)
]
df_new = spark.createDataFrame(new_data)
df_combined = df_manual.union(df_new)
df_combined.createOrReplaceTempView("people_combined")
spark.sql("SELECT * FROM people_combined").show()

# Update Age for the person with ID = 2
updated_query = """
SELECT ID, Name,
    CASE
        WHEN ID = 2 THEN 32
        ELSE Age
    END AS Age
FROM people_combined
"""
df_updated = spark.sql(updated_query)
df_updated.createOrReplaceTempView("people_updated")
spark.sql("SELECT * FROM people_updated").show()

# Delete records where ID = 1
df_deleted = spark.sql("SELECT * FROM people_updated WHERE ID != 1")
df_deleted.createOrReplaceTempView("people_updated")
spark.sql("SELECT * FROM people_updated").show()


+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  2|    Bob| 30|
|  3|Charlie| 35|
+---+-------+---+

+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 30|
|  3|Charlie| 35|
|  4|  David| 40|
|  5|    Eva| 28|
+---+-------+---+

+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  1|  Alice| 25|
|  2|    Bob| 32|
|  3|Charlie| 35|
|  4|  David| 40|
|  5|    Eva| 28|
+---+-------+---+

+---+-------+---+
| ID|   Name|Age|
+---+-------+---+
|  2|    Bob| 32|
|  3|Charlie| 35|
|  4|  David| 40|
|  5|    Eva| 28|
+---+-------+---+

